# Проверка значимости для определённых метрик

## Пропорциональные (качественные) метрики

Проверка размера тестовой группы для желаемого минимального статистически значимого изменения в хи-квадрат. Используется для проверки статзначимости для пропорциональных (качественных) метрик

In [1]:
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import TTestIndPower

baseline_cr = 0.2 # базовый уровень 
min_effect = 0.05 # минимальный значимый результат

effect_size = proportion_effectsize(baseline_cr, baseline_cr + min_effect)

alpha = 0.05 # уровень значимости
power = 0.8  #уровень мощности
power_analysis = TTestIndPower()
sample_size = power_analysis.solve_power(effect_size, power=power, alpha=alpha, alternative='two-sided')

print(f"Необходимый размер выборки: {sample_size:.0f}")

Необходимый размер выборки: 1093


C:\Users\Zver\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:6832: RuntimeWarning: invalid value encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
C:\Users\Zver\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:6826: RuntimeWarning: invalid value encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


Сам тест

In [2]:
import numpy as np
import scipy.stats as stats

#данные
group_A = [50, 100]
group_B = [60, 90]

# тест
chi2, p, dof, ex = stats.chi2_contingency([group_A, group_B], correction=False)

# доверительный интервал для изменения
lift = (group_B[0]/group_B[1])/(group_A[0]/group_A[1])
std_error = np.sqrt(1/group_B[0] + 1/group_B[1] + 1/group_A[0] + 1/group_A[1])
ci = stats.norm.interval(0.95, loc=lift, scale=std_error)

# результаты
print("Хи-квадрат p-value: ", p)
print("Доверительный интервал изменения: ", ci)

# вывод уровня значимости
if p < 0.05 and ci[0] > 1:
    print("Вариант лучше.")
else:
    print("Разницы нет.")

Хи-квадрат p-value:  0.23088408514416142
Доверительный интервал изменения:  (0.8622166237118749, 1.8044500429547916)
Разницы нет.


## Количественные

Здесь нужно использовать метод CUPED, для того, чтоб нивелировать большую дисперсию

In [3]:
import pandas as pd

def get_cuped_adjusted(A_before, B_before, A_after, B_after):
    cv = cov([A_after + B_after, A_before + B_before])
    theta = cv[0, 1] / cv[1, 1]
    mean_before = mean(A_before + B_before)
    A_after_adjusted = [after - (before - mean_before) * theta for after, before in zip(A_after, A_before)]
    B_after_adjusted = [after - (before - mean_before) * theta for after, before in zip(B_after, B_before)]
    return A_after_adjusted, B_after_adjusted

После того, как мы получили данные, стоит оценить размер выборки, тут есть два варианта.

Если выборка большая…

То нужно посмотреть на нормальность распределения данных, в целом все просто, проверить нормальность распределения поможет вот этот скрипт, который использует тест Шапиро‑Уилка, он подойдет в большинстве случаев:

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro, norm

data = pd.read_csv('adjusted_experiment_data.csv')

# Применяем тест Шапиро-Уилка
stat, p = shapiro(data)

alpha = 0.05
if p > alpha:
    print("Нормальное распределение.")
else:
    print("Не нормальное распределение.")

# График с расрпеделением
fig, ax = plt.subplots()
ax.hist(data, bins=5, density=True, alpha=0.5, label='Data')

mu, std = norm.fit(data)
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
ax.plot(x, p, 'k', linewidth=2, label='Normal distribution')

FileNotFoundError: [Errno 2] No such file or directory: 'adjusted_experiment_data.csv'

Дальше проводим t‑test, наиболее известным является т‑критерий стъюдента, он также подойдет. Если же между группами осталась неравная дисперсия, то лучше подойдет т‑критерий Уэлча, в целом может использовать его всегда не будет ошибкой, но вы можете столкнуться с меньшим уровнем значимости.

In [5]:
import pandas as pd
import scipy.stats as stats

data = pd.read_csv('adjusted_experiment_data.csv')

control = data[data['group_type'] == 'control']['experiment_data']
test = data[data['group_type'] == 'test']['experiment_data']

# т-критерий Уэлча
welch_t, welch_p = stats.ttest_ind(control, test, equal_var=False)

# т-критерий Стъюдента
student_t, student_p = stats.ttest_ind(control, test, equal_var=True)


print("Welch's t-test:")
print("t-statistic: ", welch_t)
print("p-value: ", welch_p)

print("\nStudent's t-test:")
print("t-statistic: ", student_t)
print("p-value: ", student_p)

FileNotFoundError: [Errno 2] No such file or directory: 'adjusted_experiment_data.csv'